# CSULA Data Science (CS4661)
### Yunjian Lu
### Final Project
### House prices prediction


In [178]:
# import all methods 
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import  statistics
from sklearn import preprocessing


## Data Gethering:

In [179]:
# read the dataset as a CSV file directly from the book website
houses_df = pd.read_csv("final/train.csv", index_col=0)

# show the first 5 rows
houses_df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [180]:
missing = houses_df.isnull().sum()
print(missing)

MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
Street             0
                ... 
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
SalePrice          0
Length: 80, dtype: int64


## Data Cleaning and Numerization 

In [181]:
"""
Filling all  missing numberical values with the columns's median or most common sting
"""
miss_count = houses_df.isnull().sum()
miss_column = miss_count[miss_count > 0]
for column in list(miss_column.index):
    # fill with most common sting
    if houses_df[column].dtype == 'object':
        houses_df[column].fillna(houses_df[column].value_counts().index[0], inplace=True)
    # fill with median value of that column
    elif houses_df[column].dtype == 'int64' or 'float64' or 'int16' or 'float16':
        houses_df[column].fillna(houses_df[column].median(), inplace=True)
# print the filled values
houses_df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,Grvl,Reg,Lvl,AllPub,Inside,...,0,Gd,MnPrv,Shed,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,Grvl,Reg,Lvl,AllPub,FR2,...,0,Gd,MnPrv,Shed,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,Grvl,IR1,Lvl,AllPub,Inside,...,0,Gd,MnPrv,Shed,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,Grvl,IR1,Lvl,AllPub,Corner,...,0,Gd,MnPrv,Shed,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,Grvl,IR1,Lvl,AllPub,FR2,...,0,Gd,MnPrv,Shed,0,12,2008,WD,Normal,250000


In [182]:
"""
Conveting all string value(Non-numeric) into numerical values
"""
# get all the non-numeric data
non_numeric_cols = list(houses_df.select_dtypes(exclude=[np.number]).columns)
# get all the non-numeric indexes
non_numeric_cols_indexes  = []
for col in non_numeric_cols:
    non_numeric_cols_indexes.append(houses_df.columns.get_loc(col))

# Conveting the Non-numeric values to numerical values   
label_enc = LabelEncoder()
for i in non_numeric_cols_indexes:
    houses_df.iloc[:,i] = label_enc.fit_transform(houses_df.iloc[:,i])



In [183]:
houses_df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,3,65.0,8450,1,0,3,3,0,4,...,0,2,2,2,0,2,2008,8,4,208500
2,20,3,80.0,9600,1,0,3,3,0,2,...,0,2,2,2,0,5,2007,8,4,181500
3,60,3,68.0,11250,1,0,0,3,0,4,...,0,2,2,2,0,9,2008,8,4,223500
4,70,3,60.0,9550,1,0,0,3,0,0,...,0,2,2,2,0,2,2006,8,0,140000
5,60,3,84.0,14260,1,0,0,3,0,2,...,0,2,2,2,0,12,2008,8,4,250000


## Features Selection and Model Training:

In [184]:
# Set all fetures to X
features_cols = list(houses_df.loc[:, houses_df.columns != "SalePrice"])
X = houses_df[features_cols]

# check the size:
print(X.shape)

# show the first 5 rows
X.head()

(1460, 79)


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
1,60,3,65.0,8450,1,0,3,3,0,4,...,0,0,2,2,2,0,2,2008,8,4
2,20,3,80.0,9600,1,0,3,3,0,2,...,0,0,2,2,2,0,5,2007,8,4
3,60,3,68.0,11250,1,0,0,3,0,4,...,0,0,2,2,2,0,9,2008,8,4
4,70,3,60.0,9550,1,0,0,3,0,0,...,0,0,2,2,2,0,2,2006,8,0
5,60,3,84.0,14260,1,0,0,3,0,2,...,0,0,2,2,2,0,12,2008,8,4


In [186]:
# Set SalePrice as the y axis
y = houses_df['SalePrice']

# checking the size of target vector:
y.size
print(y)

Id
1       208500
2       181500
3       223500
4       140000
5       250000
         ...  
1456    175000
1457    210000
1458    266500
1459    142125
1460    147500
Name: SalePrice, Length: 1460, dtype: int64


In [187]:
# Splitting the dataset into testing and training:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, 
                                                    random_state=2)

# Instantiate LinearRegression model
my_linreg = LinearRegression()

#Ttrain the model to the with data:
my_linreg.fit(X_train, y_train)

LinearRegression()

## Prediction :

In [188]:
# make predictions on the testing set
y_prediction = my_linreg.predict(X_test)

print(y_prediction)

[259910.82707932 186913.52990752 281828.07238481 209518.37407181
 221227.69427    187480.25571491 218108.15198854 208780.35893849
 128564.56177172 371544.58863077 252787.4307921  221096.64587813
 126714.24639695 148354.38711263  88428.34382751 221250.75144362
 259158.01663607 284433.37855158 228820.48052669 310181.08950847
 360710.6979296  169059.99249644 182537.63387926 220862.33622512
 405233.84860605 246864.79024047 146047.45598027 268604.78318593
 218742.74669926 429630.40528137 319853.56718584 179928.12552101
 259828.41978066 116009.35700792 181608.18202091 219162.17461027
 131396.7108442  293628.62485057 338121.39305183 372187.58826962
 165714.09770908 229868.96385543 152689.70990923  88545.22899279
  89130.83572888 257812.57332742  61148.88737126 138369.68449023
 102505.59957482 152348.43221997 157809.05836687 104212.56704942
 230233.32584864 164160.74833792 139511.65126522 285642.50044963
 130276.65867652 251074.05006763 383669.41618536 116315.77363015
 163771.95399718 133067.3

## Evaluation by MSE and 10 fold Cross-Validation:


In [189]:
# Calculating "Mean Square Error" (MSE):
mse = metrics.mean_squared_error(y_test, y_prediction)

# Using numpy sqrt function to take the square root and calculate "Root Mean Square Error" (RMSE)
rmse = np.sqrt(mse)

print(rmse)

57283.51819481026


In [190]:
# calculating rmse with 10 fold cross-validation
mse_list = cross_val_score(my_linreg, X, y, cv=10, scoring='neg_mean_squared_error')

mse_list_positive = -mse_list

# using numpy sqrt function to calculate rmse:
rmse_list = np.sqrt(mse_list_positive)
print(rmse_list)

# calculate the average RMSE as final result of cross validation:
print("Mean RMSE:", rmse_list.mean())

[22359.60130692 34826.39349809 25990.52813128 42101.93166586
 36055.30364137 29001.10602645 28407.27261137 26868.29535287
 66593.11734911 34195.09238845]
Mean RMSE: 34639.864197177914


In [191]:
# normalize (scale) the features.
scaled_X = pd.DataFrame()
for column in X.columns:
    normalized_arr = preprocessing.normalize([X[column]])
    scaled_X[column] = normalized_arr[0]

scaled_X.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,0.022151,0.025376,0.023223,0.015255,0.026225,0.0,0.032721,0.027394,0.0,0.030544,...,0.0,0.0,0.026203,0.026975,0.026191,0.0,0.007613,0.026174,0.027292,0.026652
1,0.007384,0.025376,0.028582,0.017331,0.026225,0.0,0.032721,0.027394,0.0,0.015272,...,0.0,0.0,0.026203,0.026975,0.026191,0.0,0.019032,0.026161,0.027292,0.026652
2,0.022151,0.025376,0.024295,0.020310,0.026225,0.0,0.000000,0.027394,0.0,0.030544,...,0.0,0.0,0.026203,0.026975,0.026191,0.0,0.034258,0.026174,0.027292,0.026652
3,0.025843,0.025376,0.021437,0.017241,0.026225,0.0,0.000000,0.027394,0.0,0.000000,...,0.0,0.0,0.026203,0.026975,0.026191,0.0,0.007613,0.026148,0.027292,0.000000
4,0.022151,0.025376,0.030011,0.025743,0.026225,0.0,0.000000,0.027394,0.0,0.015272,...,0.0,0.0,0.026203,0.026975,0.026191,0.0,0.045678,0.026174,0.027292,0.026652


In [192]:
# retrain the model with scaled data :
# Set all fetures to X
features_cols = list(houses_df.loc[:, houses_df.columns != "SalePrice"])
all_fea_rmse = []
for ele in features_cols:
    X = (houses_df[[ele]])

# X = houses_df[features_cols]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, 
                                                        random_state=2)

    my_linreg = LinearRegression()
    my_linreg.fit(X_train, y_train)
    y_prediction = my_linreg.predict(X_test)
    mse = metrics.mean_squared_error(y_test, y_prediction)
    rmse = np.sqrt(mse)
    all_fea_rmse.append(rmse)


In [193]:
print(statistics.mean(all_fea_rmse),)

80331.52690924447


In [194]:
"""Thank You"""

'Thank You'